In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
import pandas as pd
import time

# Configure Chrome to avoid detection
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--start-maximized")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")



driver = webdriver.Chrome(options=chrome_options)
genres = ["biography","crime","family","sci-fi","music"]  # add more if needed

# Final DataFrame to store all results
final_df = pd.DataFrame()

for genre in genres:
    url = f"https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31&genres={genre}"
    driver.get(url)
    time.sleep(5)

    def click_load_more():
        try:
            load_more_button = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/div[2]/div/span/button/span/span')
            ActionChains(driver).move_to_element(load_more_button).perform()
            load_more_button.click()
            time.sleep(5)
            return True
        except Exception as e:
            print("No more content to load or error:", e)
            return False

    while click_load_more():
        print("Clicked 'Load More' button")

    print("✅ Finished loading all movies for", genre)
    #//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/ul/li[1]/div/div/div/div[1]/div[2]/span/div/span
    #//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/ul/li[1]/div/div/div
    #//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/ul/li[1]./div/div/div/div[1]/div[2]/div[1]
    titles = []
    ratings = []
    votings = []
    durations = []

    movie_items = driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/ul/li')

    for movie_item in movie_items:
        try:
            title = movie_item.find_element(By.XPATH, './div/div/div/div[1]/div[2]/div[1]/a/h3').text
            rating = movie_item.find_element(By.XPATH, './div/div/div/div[1]/div[2]/span/div/span/span[1]').text
            voting = movie_item.find_element(By.XPATH, './div/div/div/div[1]/div[2]/span/div/span/span[2]').text
            duration = movie_item.find_element(By.XPATH, './div/div/div/div[1]/div[2]/div[2]/span[2]').text

            titles.append(title)
            ratings.append(rating)
            votings.append(voting)
            durations.append(duration)

        except Exception as e:
            print(f"Error extracting data for a movie: {e}")
            continue

    df = pd.DataFrame({
        'Title': titles,
        'Rating': ratings,
        'Votes': votings,
        'Duration': durations,
        'Genre': genre
    })
df.to_csv(f"{genre}_2024_movies.csv", index=False)
final_df = pd.concat([final_df, df], ignore_index=True)

# Save combined CSV

final_df.to_csv("overall_genres_2024_movies.csv", index=False)
print("\n All genres saved to all_genres_2024_movies.csv")


print("Done!")
driver.quit()

Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
No more content to load or error: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/div[2]/div/span/button/span/span"}
  (Session info: chrome=135.0.7049.115); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7A1B6EFA5+77893]
	GetHandleVerifier [0x00007FF7A1B6F000+77984]
	(No symbol) [0x00007FF7A19391BA]
	(No symbol) [0x00007FF7A198F16D]
	(No symbol) [0x00007FF7A198F41C]
	(No symbol) [0x00007FF7A19E2237]
	(No symbol) [0x00007FF7A19B716F]
	(No symbol) [0x00007FF7A19DF07F]
	(No s

In [15]:
df.to_csv("sci-fi.csv")

In [16]:
import pandas as pd

# First read the file as plain text to remove quotes
with open('sci-fi.csv', 'r', encoding='utf-8') as f:
    content = f.read().replace('"', '')

# Save the cleaned content to a temporary file
with open('sci-fi_cleaned.csv', 'w', encoding='utf-8') as f:
    f.write(content)

# Now read with flexible column handling
df = pd.read_csv('temp_cleaned.csv', engine='python', on_bad_lines='warn')

# Optional: Check for inconsistent rows
print("Rows with inconsistent columns:", df[df.isnull().any(axis=1)])

FileNotFoundError: [Errno 2] No such file or directory: 'temp_cleaned.csv'

In [17]:
import pandas as pd

genre_files = ['biography_cleaned.csv', 'crime_cleaned.csv', 'family_cleaned.csv', 'music_cleaned.csv', 'sci-fi_cleaned.csv']

dataframes = []
for file in genre_files:
    try:
        df = pd.read_csv(file, on_bad_lines='skip')
        if df.empty:
            print(f"Warning: {file} is empty or contains only headers.")
        else:
            dataframes.append(df)
            print(f'Successfully read {file}')
    except FileNotFoundError:
        print(f"Error: {file} not found. Check the file path.")
    except pd.errors.EmptyDataError:
        print(f"Error: {file} is empty.")
    except pd.errors.ParserError as e:
        print(f"Error parsing {file}: {e}")
    except Exception as e:
        print(f"Error reading {file}: {e}")

# Check if any DataFrames were successfully read
if not dataframes:
    raise ValueError("No DataFrames to concatenate. Check the CSV files for errors.")

Successfully read biography_cleaned.csv
Successfully read crime_cleaned.csv
Successfully read family_cleaned.csv
Successfully read music_cleaned.csv
Successfully read sci-fi_cleaned.csv


In [18]:
# Combine the DataFrames
combined_df = pd.concat(dataframes, ignore_index=True)

# Save the combined DataFrame to a single CSV file (optional)
combined_df.to_csv('overall_movies.csv', index=False)
print('Combined data saved to overall_movies.csv')

Combined data saved to overall_movies.csv


In [19]:
import mysql.connector

connection = mysql.connector.connect(
  host = "gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
  port = 4000,
  user = "43ar8iQYBuuTJzh.root",
  password = "<zhpR0aG2uWbHhVPh>",
)
mycursor = connection.cursor(buffered=True)


ProgrammingError: 1045 (28000): Access denied for user '43ar8iQYBuuTJzh.root'@'223.185.27.170' (using password: YES)